In [1]:
from pathlib import Path
from pl_models.DEA import DAE_LitModel
import matplotlib.pyplot as plt
import nibabel as nib
import torch
from torchvision import transforms
from BIDS import NII
from BIDS.core.np_utils import np_dice
import numpy as np
from BIDS.core.np_utils import np_map_labels, Label_Map
from sklearn.manifold import TSNE
import torchvision
from dataloader.dataset_factory import get_data_loader, get_dataset
from torch.utils.data import DataLoader
from utils import arguments
import tqdm

opt = arguments.DAE_Option().get_opt(None, None)


dataset = get_dataset(arguments.DataSet_Option,"train")

#part_tr = torch.utils.data.random_split(dataset, [0, len(dataset)-10])[0]
#subset_indices = range(100)#[0,1,2,3,4,5,6,7,8,9]
part_tr = dataset #torch.utils.data.Subset(dataset, subset_indices)
#datset = dataset.
val_dataloader = DataLoader(
        part_tr,
        batch_size=1,#opt.batch_size,
        #sampler=sampler,
        # with sampler, use the sample instead of this option
        shuffle=False,# if sampler else shuffle,
        num_workers=16,#opt.num_cpu,
        pin_memory=True,
        #drop_last=drop_last,
        #multiprocessing_context=get_context("fork"),
        persistent_workers=True,
    )
#/media/DATA/martina_ma/dae/lightning_logs/DAE_3D_95_old_verse_w_norm/version_11/checkpoints/epoch=0-step=1488_d_score_latest.ckpt
#/media/DATA/martina_ma/dae/lightning_logs/DAE_3D_95_old_verse_w_norm/version_22/checkpoints/epoch=40-step=61008d_score=0.9937_d_score_latest copy.ckpt
#/media/DATA/martina_ma/dae/lightning_logs/DAE_3D_95_old_verse_w_norm/version_30/checkpoints/epoch=10-step=46717d_score=0.9912_d_score_latest.ckpt
#/media/DATA/martina_ma/dae/lightning_logs/DAE_3D_95_old_verse_w_norm/version_30/checkpoints/epoch=61-step=261630_latest.ckpt
checkpoint_path ="/media/DATA/martina_ma/dae/lightning_logs/DAE_3D_95_old_verse_w_norm/version_30/checkpoints/epoch=10-step=46717d_score=0.9912_d_score_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_133/checkpoints/epoch=3-step=310_latest.ckpt"#/media/DATA/martina_ma/dae/lightning_logs/DAE_3D_95_old_verse_w_norm/version_10/checkpoints/epoch=0-step=1488_d_score_latest copy.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_232/checkpoints/epoch=283-step=27190_latest.ckpt"# "/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_133/checkpoints/epoch=3-step=310_latest.ckpt"
device = "cuda:0"  # or "cpu" if you want to use CPU
assert Path(checkpoint_path).exists()
model = DAE_LitModel.load_from_checkpoint(checkpoint_path)
model.ema_model.eval()
model.ema_model.to(device)
# outputs = np.empty((2, 512))
# i = 0
# for batch in val_dataloader:
#     # print(i)
#     l = batch['label']
#     images = batch['img'].to(device)
 
#     output = model.encode(images)
 
#     current_outputs = output.detach().cpu().numpy()
#     features = np.concatenate((outputs, current_outputs))
#     i = i+1
    
    
# print(i)
#print(features)
# batch_imgs = next(iter(val_dataloader))
# imgs = batch_imgs['img']  # Extract 'img' from the dictionary
# embs = model.encode(imgs.to(device))  # Ensure the images are on the correct device


# print("Input images: " + str(imgs.shape))
# print("Embeddings: " + str(embs.shape))



/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/media/DATA/martina_ma/cutout/train_3D_95_all_dataset.csv
Model params: 1.27 M


Diffusion_Autoencoder_Model(
  (time_embed): TimeStyleSeperateEmbed(
    (time_embed): Sequential(
      (0): Linear(in_features=8, out_features=512, bias=True)
      (1): SiLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
    )
    (style): Identity()
  )
  (input_blocks): ModuleList(
    (0): TimestepEmbedSequential(
      (0): Conv3d(1, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    )
    (1): TimestepEmbedSequential(
      (0): ResBlock(
        (in_layers): Sequential(
          (0): GroupNorm32(8, 8, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Conv3d(8, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        )
        (h_upd): Identity()
        (x_upd): Identity()
        (emb_layers): Sequential(
          (0): SiLU()
          (1): Linear(in_features=512, out_features=16, bias=True)
        )
        (cond_emb_layers): Sequential(
          (0): SiLU()
          (1): Linear(in_features=512, out_features=8,

In [1]:
import pandas as pd
import plotly.express as px
from sklearn.manifold import TSNE
import re
from BIDS.core import vert_constants
from BIDS.core.np_utils import np_volume
from dataloader.datasets.dataset_csv import model_map_to_segmentation_map, segmentation_map_to_model_map 
#from utils.metadata import make_anomaly_dict, add_fxclass_fracture_anomaly, parse_string
import utils.metadata as m
# Remove square brackets
# cleaned_string = original_string.strip("[]")
#m.parse_string('s')
# # Remove single quotes
# cleaned_string = cleaned_string.replace("'", "")

def add_anomaly_flag(emb_df):
    anomaly_dict = m.make_anomaly_dict()
    #print(type(anomaly_dict))
    emb_df["fracture_flag"] = 'NF'
    
    for index, row in emb_df.iterrows():
        anomaly_id = row['subject'].strip("[]").replace("'", "")
        print(type(anomaly_id))
        #print(anomaly_id)
        label = row['label']
        a = anomaly_dict.get(anomaly_id, -1)
        #print(type(a.main_fracture))
        #print(type(label))
        if a == -1:
            continue
        #print(label == a.main_fracture)
        if int(label) == a.main_fracture:
             emb_df.at[index, 'fracture_flag'] = 'F'
             print('fracture detected')
    return emb_df

def extend_anomaly_flag_fxclass(emb_df):
    #emb_df["fracture_flag"] = 'NF'

    anomaly_dict_fx = m.add_fxclass_fracture_anomaly()
    for index, row in emb_df.iterrows():
        anomaly_id = row['subject'].strip("[]").replace("'", "")
        print(type(anomaly_id))
        #print(anomaly_id)
        label = row['label']
        fractures_list = anomaly_dict_fx.get(anomaly_id, -1)
        #print(type(a.main_fracture))
        #print(type(label))
        if fractures_list == -1:
            continue
        #print(label == a.main_fracture)
        if int(label) in fractures_list:
             emb_df.at[index, 'fracture_flag'] = 'F'
             print('fracture detected')
    return emb_df
# Example DataFrame creation
data = {'subject': ['fxclass0329'],
        'label': '10'}
emb_df = pd.DataFrame(data)
r = extend_anomaly_flag_fxclass(emb_df)
print(r)


####################################
/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io

<class 'str'>
fracture detected
       subject label fracture_flag
0  fxclass0329    10             F


####################################
/media/DATA/martina_ma/dae/utils/metadata.py:46: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  person_id = extract_fxclass(row[0])
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/ipyke

In [3]:

def calc_volume(seg_arr):
    seg = model_map_to_segmentation_map(seg_arr)
    labels = np.unique(seg_arr)
    labels = model_map_to_segmentation_map(labels)[1:]
    vol_dict = np_volume(seg, labels)
    total_volume = sum(vol_dict.values())
    return total_volume

# Input string match = re.search(r'\b\d+\b', string)
def extract_nr(pattern_string):
    # Use re.search to find the match in the string
    match = re.search(r'\b\d+\b', pattern_string)
    # If no match is found, try a modified pattern
    if not match:
        match = re.search(r'\b\d+\b', re.sub(r'[^\d]+', '', pattern_string))
    # Check if a match is found
    if match:
        # Extract the matched number
        extracted_number = match.group(0)
        #print("Extracted Number:", extracted_number)
    else:
        print("No match found.")
    return extracted_number

v_idx2name = {
     1: "C1",     2: "C2",     3: "C3",     4: "C4",     5: "C5",     6: "C6",     7: "C7", 
     8: "T1",     9: "T2",    10: "T3",    11: "T4",    12: "T5",    13: "T6",    14: "T7",    15: "T8",    16: "T9",    17: "T10",   18: "T11",   19: "T12", 28: "T13",
    20: "L1",    21: "L2",    22: "L3",    23: "L4",    24: "L5",    25: "L6",    
    26: "S1",    29: "S2",    30: "S3",    31: "S4",    32: "S5",    33: "S6",
    27: "Cocc"
}

def get_region(label):
    if label in [1,2,3,4,5,6,7]:#range(1, 8):
        return 'C'
    elif label in [8,9,10,11,12,13,14,15,16,17,18,19,28]:#range(8, 20):
        return 'T'
    elif label in range(20, 26):
        return 'L'
    elif label in [26,29,30,31,32,33]:#range(26, 34):
        return 'S'
    elif label == 27:
        return 'Cocc'
    #elif label in vert_constants.subreg_idx2name:
     #   return 'subreg'
    else:
        return 'unknown'


In [11]:

def extract_embeddings(dataloader, model):
    data_list = []

    with torch.no_grad():
        for x in dataloader:
            img = x['img'].to(device)
            label = x['label']
            subject = x['subject']
            
            output = model.encode(img)

            
            label_str = str(label)
            subject_str = str(subject)
            #print(subject_str)
            # if (int(extract_nr(subject_str)) == 526 and int(extract_nr(label)) == 25):# or (subject_str == "['verse534']" and label == "['025']") or (subject_str == "['verse617']" and label == "['024']"):
            #     print("skipped")
            #     continue 
            region = get_region(int(extract_nr(label_str)))
            volume = calc_volume(img.cpu().numpy())

            data_item = {
                "embeddings": output,  
                "label": label_str,
                "subject": subject_str,
                "region": region,
                "volume": volume
            }

            data_list.append(data_item)

    # Convert the list of dictionaries to a DataFrame
    dataframe = pd.DataFrame(data_list)

    return dataframe


# Example usage:
embeddings_dataframe = extract_embeddings(val_dataloader, model)


KeyboardInterrupt: 

In [27]:


#print(embeddings_dataframe['fracture_flag']=='F')
# Extract features and labels
features = embeddings_dataframe["embeddings"].tolist()
labels = embeddings_dataframe["label"].tolist()
volumes = embeddings_dataframe["volume"].tolist()
#print(embeddings_dataframe)
#embeddings_dataframe['label'] = [int(extract_nr(label)) for label in labels]
#print(embeddings_dataframe)
embeddings_dataframe = add_anomaly_flag(embeddings_dataframe)
embeddings_dataframe = extend_anomaly_flag_fxclass(embeddings_dataframe)
#print(embeddings_dataframe)
# Concatenate tensors along dimension 0
features_tensor = torch.cat(features, dim=0)

# Convert the concatenated tensor to a NumPy array
features_array = features_tensor.cpu().numpy()


# print(labels)
# numerical_labels = [int(extract_nr(label)) for label in labels]
# print(numerical_labels)
# reg_labels = [get_region(label) for label in numerical_labels]


<class 'str'>
<class 'str'>
fracture detected
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
fracture detected
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'

AttributeError: module 'utils.metadata' has no attribute 'add_fxclass_fracture_anomaly'

In [ ]:
from sklearn.decomposition import PCA

import plotly.express as px

# Assuming `features_array` is your embeddings array with shape (num_samples, num_features)

# Create a PCA instance with 3 components
num_components = 2
pca = PCA(n_components=num_components)

# Fit the PCA model and transform the data
pca_result = pca.fit_transform(features_array)

# Create a DataFrame for Plotly Express
pca_df = pd.DataFrame(data={"PC1": pca_result[:, 0], "PC2": pca_result[:, 1]})

# Plot 3D scatter plot with Plotly Express
fig = px.scatter(pca_df, x='PC1', y='PC2', title="PCA of Embeddings (2 Components)")
fig.show()

In [14]:
from sklearn.decomposition import PCA

import plotly.express as px

# Assuming `features_array` is your embeddings array with shape (num_samples, num_features)

# Create a PCA instance with 3 components
num_components = 50
pca = PCA(n_components=num_components)

# Fit the PCA model and transform the data
pca_result = pca.fit_transform(features_array)

# Apply t-SNE
tsne = TSNE(n_components=2, perplexity=10, n_iter=1500)
tsne_result = tsne.fit_transform(features_array)#tsne.fit_transform(features_array)

# Add t-SNE results to the DataFrame
embeddings_dataframe["tsne_1"] = tsne_result[:, 0]
embeddings_dataframe["tsne_2"] = tsne_result[:, 1]
#embeddings_dataframe["tsne_3"] = tsne_result[:, 2]
# Plot using plotly.express.scatter
# fig = px.scatter(embeddings_dataframe, x="tsne_1", y="tsne_2", color="label", hover_data=embeddings_dataframe["subject"], title="t-SNE of Embeddings")
# fig.show()

In [ ]:
# Apply t-SNE
tsne_3D = TSNE(n_components=3, perplexity=30, n_iter=1500)
tsne_result_3D = tsne_3D.fit_transform(features_array)

# Add t-SNE results to the DataFrame
embeddings_dataframe["tsne_1_3D"] = tsne_result_3D[:, 0]
embeddings_dataframe["tsne_2_3D"] = tsne_result_3D[:, 1]
embeddings_dataframe["tsne_3_3D"] = tsne_result_3D[:, 2]
numerical_labels = [int(extract_nr(label)) for label in labels]
# Plot 3D scatter plot with Plotly Express
fig = px.scatter_3d(embeddings_dataframe, x=tsne_result_3D[:, 0], y=tsne_result_3D[:, 1], z=tsne_result_3D[:, 2],color=labels, title="Tsne of Embeddings (3 Components)")
fig.show()

In [17]:
# Add t-SNE results to the DataFrame
# embeddings_dataframe["tsne_1"] = 
# embeddings_dataframe["tsne_2"] = 


emb_df = embeddings_dataframe#[embeddings_dataframe['volume'] > 10000]
labels = emb_df["label"].tolist()
regions = [get_region(label) for label in labels]#[get_region(int(extract_nr(label))) for label in labels]
region = regions#embeddings_dataframe["region"].tolist()
subjects = emb_df['subject'].tolist()
# Plot using plotly.express.scatter
l = False
if l:
    #fig = px.scatter_3d(embeddings_dataframe, x=tsne_result[:, 0], y=tsne_result[:, 1], z = tsne_result[:,2], color=labels,  title="t-SNE of Embeddings")
    fig = px.scatter(embeddings_dataframe, x=tsne_result[:, 0], y=tsne_result[:, 1], color=numerical_labels,hover_data=["subject"], title="t-SNE of Embeddings")
    fig.update_layout(legend_orientation="h")
    fig.show()
else: 
    emb_df_x = emb_df['tsne_1']
    emb_df_y = emb_df['tsne_2']#emb_df["region"],
    fig_reg = px.scatter(emb_df, x=emb_df_x, y=emb_df_y, color=emb_df['region'], symbol=emb_df['label'],hover_name=emb_df['fracture_flag'], hover_data=["subject"], title="t-SNE of Embeddings")
    fig_reg.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,f1_score

# Assuming embeddings_dataframe is your DataFrame containing "region" and "number" columns
regions = embeddings_dataframe["region"].tolist()
labels_all = embeddings_dataframe["label"].tolist()
numbers = [int(extract_nr(label)) for label in labels_all]

# Create a binary label indicating if  region is 'T' and the number is 13
#y = [(region == 'T') for region in  regions]
y = [(nr == 1) for nr in  numbers]
# Assuming tsne_result is your feature matrix
X = features_array#tsne_result

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X
y_train = y
y_test = y
# Create and train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.6f}")
print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1 Score: {f1:.6f}")
print("Confusion Matrix:")
print(conf_matrix)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

regions = embeddings_dataframe["region"].tolist()
X = tsne_result
y = regions

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X, y)

y_pred = model.predict(X)

f1 = f1_score(y, y_pred, average='weighted')  

print(f"F1 Score: {f1}")

In [ ]:
from pathlib import Path
from pl_models.DEA import DAE_LitModel
import matplotlib.pyplot as plt
import nibabel as nib
import torch
from torchvision import transforms
from BIDS import NII
from BIDS.core.np_utils import np_dice
import numpy as np
from BIDS.core.np_utils import np_map_labels, Label_Map
from sklearn.manifold import TSNE
import torchvision
from dataloader.dataset_factory import get_data_loader, get_dataset
from torch.utils.data import DataLoader
from utils import arguments
import tqdm

opt = arguments.DAE_Option().get_opt(None, None)


dataset = get_dataset(arguments.DataSet_Option,"train")

#part_tr = torch.utils.data.random_split(dataset, [0, len(dataset)-10])[0]
#subset_indices = range(100)#[0,1,2,3,4,5,6,7,8,9]
part_tr = dataset #torch.utils.data.Subset(dataset, subset_indices)
#datset = dataset.
val_dataloader = DataLoader(
        part_tr,
        batch_size=1,#opt.batch_size,
        #sampler=sampler,
        # with sampler, use the sample instead of this option
        shuffle=False,# if sampler else shuffle,
        num_workers=16,#opt.num_cpu,
        pin_memory=True,
        #drop_last=drop_last,
        #multiprocessing_context=get_context("fork"),
        persistent_workers=True,
    )

def extract_volume(dataloader):
    data_list = []

    with torch.no_grad():
        for x in dataloader:
            img = x['img']#.to(device)
            label = x['label']
            subject = x['subject']
            
            label_str = str(label)
            subject_str = str(subject)
            #print(subject_str)
            # if (int(extract_nr(subject_str)) == 526 and int(extract_nr(label)) == 25):# or (subject_str == "['verse534']" and label == "['025']") or (subject_str == "['verse617']" and label == "['024']"):
            #     print("skipped")
            #     continue 
            region = get_region(int(extract_nr(label_str)))
            volume = calc_volume(img.numpy())

            data_item = {
                "label": label_str,
                "subject": subject_str,
                "region": region,
                "volume": volume
            }

            data_list.append(data_item)

    # Convert the list of dictionaries to a DataFrame
    dataframe = pd.DataFrame(data_list)

    return dataframe

df = extract_volume(val_dataloader)
df_filtered = df[df['volume'] >= 8000]
df_outliers = df[df['volume'] <= 8000]
# Display the filtered DataFrame
print(df_filtered.head(20))
print(df_filtered.describe())

In [ ]:
from scipy import stats
z = np.abs(stats.zscore(df_diabetics['age']))
print(z)
emb_df = embeddings_dataframe#[embeddings_dataframe['volume'] > 10000]
fig_reg = px.scatter(emb_df, x=emb_df['volume'], color=emb_df['region'], symbol=emb_df['label'],hover_data=["subject"], title="t-SNE of Embeddings")
fig_reg.show()

In [ ]:
from scipy import stats
import pandas as pd

def find_outliers_IQR( column_name,label):
    df = embeddings_dataframe[embeddings_dataframe['label'].str.contains(label)]
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    IQR = q3 - q1

    outliers = df[((df[column_name] < (q1 - 1.5 * IQR)) | (df[column_name] > (q3 + 1.5 * IQR)))]

    return outliers

# Assuming embeddings_dataframe is your DataFrame
#e_df = embeddings_dataframe[embeddings_dataframe['label'].str.contains('001')]

# Call the function with the DataFrame and the column name 'volume'
outliers = find_outliers_IQR( 'volume', '001')
print("001", outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '002')
print("002",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '003')
print("003",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '004')
print("004",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '005')
print("005",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '006')
print("006",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '007')
print("007",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '008')
print("008",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '009')
print("009",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '010')
print("010",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '011')
print("011",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '012')
print("012",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '013')
print("013",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '014')
print("014",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '015')
print("015",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '016')
print("016",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '017')
print("017",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '018')
print("018",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '019')
print("019",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '020')
print("020",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '021')
print("021",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '022')
print("022",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '023')
print("023",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '024')
print("024",outliers[['subject', 'volume']])
outliers = find_outliers_IQR( 'volume', '025')
#outliers = find_outliers_IQR( 'volume', '027')
# Display the outliers with 'subject'
print("025",outliers[['subject', 'volume']])

#z = np.abs(stats.zscore(embeddings_dataframe['volume']))
#print(z)
def find_outliers_IQR(df):
#https://careerfoundry.com/en/blog/data-analytics/how-to-find-outliers/
   
   q1=df.quantile(0.25)

   q3=df.quantile(0.75)

   IQR=q3-q1

   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

   return outliers
#print((embeddings_dataframe['label']))
e_df = embeddings_dataframe[embeddings_dataframe['label'].str.contains('001')]
#df = df.sort_values(by=['P'], ascending=False)
outliers = find_outliers_IQR(e_df["volume"])
outliers


In [ ]:
df = embeddings_dataframe[embeddings_dataframe['label'].str.contains('001')]
fig = px.histogram(df, x='volume',color='subject')

fig.show()

In [ ]:

emb_df = embeddings_dataframe#[embeddings_dataframe['volume'] > 10000]
fig_reg = px.scatter(emb_df, x=emb_df['volume'], color=emb_df['region'], symbol=emb_df['label'],hover_data=["subject"], title="t-SNE of Embeddings")
fig_reg.show()

In [ ]:
import matplotlib

from BIDS.core import vert_constants
def get_region(label):
    if label in range(1, 8):
        return 'C'
    elif label in range(8, 20):
        return 'T'
    elif label in range(20, 26):
        return 'L'
    elif label in range(26, 34):
        return 'S'
    elif label == 27:
        return 'Cocc'
    elif label in vert_constants.subreg_idx2name:
        return 'subreg'
    else:
        return 'unknown'
    

region_colors = {
    'C': 'blue',
    'T': 'red',
    'L': 'green',
    'S': 'purple',
    'Cocc': 'orange',
    'subreg': 'gray'
}


# Function to plot
def plot_embeddings(embeddings, labels, title):
    # Convert labels to integers for sorting
    numerical_labels = [int(label) for label in labels]
    reg_labels = [get_region(label) for label in numerical_labels]
    
    # Map labels to colors
    colors = [region_colors[reg] for reg in reg_labels]

    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(embeddings[:, 0], embeddings[:, 1], c=colors, cmap='hsv', marker='o', s=40, edgecolor='k')

    # Create a custom legend for each label
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10) for color in set(colors)]
    labels_legend = [str(label) for label in set(numerical_labels)]
    
    plt.legend(handles, labels_legend, title='Labels', loc='lower right', bbox_to_anchor=(1.2, -0.05))
   
    plt.title(title)
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.show()

def plot_embeddings_per_label(embeddings, labels, title):
   #Convert labels to integers for sorting
    numerical_labels = [int(label) for label in labels]
    #reg_labels = [get_region(label) for label in numerical_labels]
    #print(reg_labels)
    # Map labels to colors
    #numerical_labels = [region_colors[get_region(label)] for label in numerical_labels]

    print(numerical_labels)
    # Map sorted labels to colors using a discrete color map
    #unique_labels = numerical_labels
    #label_to_color = {label: i for i, label in enumerate(unique_labels)}
    #colors = [label_to_color[label] for label in unique_labels]
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(embeddings[:, 0], embeddings[:, 1], c=numerical_labels,cmap='hsv',  marker='o', s=40, edgecolor='k')
    
    # Create a custom legend
    # handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=scatter.to_rgba(color), markersize=10) for color in numerical_labels]
     # Create a custom legend for each label
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=scatter.to_rgba(label), markersize=10) for label in set(numerical_labels)]
    labels_legend = [str(label) for label in set(numerical_labels)]
    
    plt.legend(handles, labels_legend, title='Labels', loc='lower right', bbox_to_anchor=(1.2, -0.05))
   
    plt.title(title)
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.show()

In [ ]:
# EXAMINE LATENT SPACE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
# Function to extract embeddings - TODO: Adjust the Net class to extract embeddings
def extract_embeddings(dataloader, model):
    embeddings = []
    labels = []

    with torch.no_grad():
        for x in dataloader:
            img = x['img'].to(device)
            #print(x)
            label = x['label']#.to(device)
            subject = x['subject']
            
            output = model.encode(img)
            
            embeddings.append(output)
            labels.append(label)
    
    return torch.cat(embeddings), sum(labels, [])#torch.cat(labels)


    # Map labels to colors
    # label_to_color = {label: i for i, label in enumerate(set(labels))}
    # colors = [label_to_color[label] for label in labels]

    # plt.figure(figsize=(8, 6))
    # scatter = plt.scatter(embeddings[:, 0], embeddings[:, 1], c=colors, cmap='viridis')#, alpha=0.5)
    # plt.colorbar(scatter)
    # plt.title(title)
    # plt.xlabel('Component 1')
    # plt.ylabel('Component 2')
    # plt.show()

test_embeddings, test_labels = extract_embeddings(val_dataloader, model)
print(test_labels)
print(test_embeddings)
import numpy as np
import matplotlib.pyplot as plt

# Flatten the embeddings
test_embeddings_flat = test_embeddings.cpu().numpy() #test_embeddings.view(test_embeddings.size(0), -1).cpu().numpy()

# Apply PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(test_embeddings_flat)

# Apply t-SNE
tsne = TSNE(n_components=2, perplexity=20)
tsne_result = tsne.fit_transform(test_embeddings_flat)

# Plot PCA result
#plot_embeddings(pca_result, test_labels, 'PCA of Train Embeddings')




In [ ]:
# Plot t-SNE result
plot_embeddings(tsne_result, test_labels, 't-SNE of Val Embeddings per Region')
plot_embeddings_per_label(tsne_result, test_labels, 't-SNE of Val Embeddings per Label')

In [ ]:
from pathlib import Path
from pl_models.DEA import DAE_LitModel
import matplotlib.pyplot as plt
import nibabel as nib
import torch
from torchvision import transforms
from BIDS import NII
from BIDS.core.np_utils import np_dice
import numpy as np
from BIDS.core.np_utils import np_map_labels, Label_Map
from sklearn.manifold import TSNE
import torchvision


#3D
def load_nifti(nii):
        
        from_im = nii.get_seg_array()
       #range_print(torch.from_numpy(from_im.astype(float)),41,41)
        #print(from_im[143,95,:])
        from_im = np_map_labels(arr=from_im,label_map={50: 49})
        from_im = from_im.astype(float)
        from_im = map_to_binary(from_im)
        # print(from_im[143,95,143])
        # print("from_im shape:", from_im.shape)
        # print("mapped:",np.unique(from_im))
        from_im = torch.from_numpy(from_im).unsqueeze(0).unsqueeze(0)
        #print("from_im shape:", from_im.shape)
        from_im = from_im.to(torch.float32)
        
        #print("tesnor", from_im.shape)
        return from_im
def map_to_binary(y):
        #y = y.astype(float)
        labelmap = {i: 1 for i in range(41, 50)} #  labelmap = {i: round((i - 40)/9, ndigits=2) for i in range(41, 50)}
        #print(labelmap)
        return np_map_labels(y, labelmap)

def show_image(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

nii = NII.load(
    "/media/DATA/martina_ma/cutout/verse040/verse040_018_subreg_cropped.nii.gz",
    True,
)
from_im = load_nifti(nii)

checkpoint_path ="/media/DATA/martina_ma/dae/lightning_logs/DAE_3D_binary/version_16/checkpoints/epoch=40-step=9905_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_232/checkpoints/epoch=283-step=27190_latest.ckpt"# "/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_133/checkpoints/epoch=3-step=310_latest.ckpt"
#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_135/checkpoints/epoch=27-step=2775_latest.ckpt"#/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_133/checkpoints/epoch=3-step=310_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_118/checkpoints/epoch=80-step=243_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_116/checkpoints/epoch=81-step=246_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_108/checkpoints/epoch=47-step=144_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_105/checkpoints/epoch=69-step=210_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_63/checkpoints/epoch=79-step=240_latest.ckpt"#"/media/DATA/martina_ma/dae/lightning_logs/DAE_NAKO_256/version_60/checkpoints/epoch=71-step=216_latest.ckpt"#lightning_logs/DAE_NAKO_256/version_26/checkpoints/epoch=404-step=405_latest.ckpt"
device = "cuda:0"  # or "cpu" if you want to use CPU
assert Path(checkpoint_path).exists()
model = DAE_LitModel.load_from_checkpoint(checkpoint_path)
model.ema_model.eval()
model.ema_model.to(device)

#encoder.eval()
#decoder.eval()
noise = torch.randn(1, 1, 144, 144, device=device)

with torch.no_grad():
    # calculate mean and std of latent code, generated takining in test images as inputs 
    images = from_im#, labels = iter(test_loader).next()
    images = images.to(device)
    latent = model.encode(images)
    n = model.encode_stochastic(images,latent,T=20)
    latent = latent.cpu()
    print(latent.shape)
    mean = latent.mean(dim=0)
    print(mean)
    std = (latent - mean).pow(2).mean(dim=0).sqrt()
    print(std)

    # sample latent vectors from the normal distribution
    latent = torch.randn(1, 512)*std + mean

    # reconstruct images from the random latent vectors
    latent = latent.to(device)
    img_recon = model.render(n, cond=latent,T=20)
    img_recon = img_recon.cpu()

    reconstructed_img = torch.where(img_recon < 0, torch.tensor(0.0), img_recon)
    r = reconstructed_img[0][0].numpy().copy()
    reconstructed_img =np.round(r)#model_map_to_segmentation_map(r)
    nii.set_array_(reconstructed_img).save("NewSampledFrom_verse040_018_ver16_binary_T20.nii.gz")
    # fig, ax = plt.subplots(figsize=(20, 8.5))
    # show_image(torchvision.utils.make_grid(img_recon[:100],10,5))
    # plt.show()